In [2]:
# !pip install scrapy
# !pip install nltk
# !pip install gensim
# !pip install xgboost
# !pip install pandastable


In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import datetime
import string
import json
import nltk
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt')
from nltk import pos_tag
from nltk.corpus import wordnet
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer 

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/anil/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /home/anil/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /home/anil/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/anil/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
sample_db = pd.read_csv('CoventryUni_Data_Scraped_articles.csv')
sample_db
print(f'{sample_db.shape[0]} records were scraped')

19396 records were scraped


In [3]:
scraped_db = pd.read_csv('CoventryUni_Data_Scraped_articles.csv')
scraped_db.head()

,Document_Id,Profile_Name,Title,Title_Link,Year_Published
0,1,Aliano Abbasi,Adopting industry 4.0 technologies in citizens...,https://pureportal.coventry.ac.uk/en/publicati...,18 Apr 2020
1,2,Sally Abbott,Emotional eating among adults with healthy wei...,https://pureportal.coventry.ac.uk/en/publicati...,3 Apr 2023
2,3,Sally Abbott,"""Is it time to throw out the weighing scales?""...",https://pureportal.coventry.ac.uk/en/publicati...,Jan 2023
3,4,Sally Abbott,Variation and outcomes of liver-reducing dieta...,https://pureportal.coventry.ac.uk/en/publicati...,Feb 2023
4,5,Sally Abbott,Global variations in preoperative practices co...,https://pureportal.coventry.ac.uk/en/publicati...,Jul 2022


In [4]:
sample_db.head(7)
#ids = scraped_db["Title"]
#scraped_db[ids.isin(ids[ids.duplicated()])]

,Document_Id,Profile_Name,Title,Title_Link,Year_Published
0,1,Aliano Abbasi,Adopting industry 4.0 technologies in citizens...,https://pureportal.coventry.ac.uk/en/publicati...,18 Apr 2020
1,2,Sally Abbott,Emotional eating among adults with healthy wei...,https://pureportal.coventry.ac.uk/en/publicati...,3 Apr 2023
2,3,Sally Abbott,"""Is it time to throw out the weighing scales?""...",https://pureportal.coventry.ac.uk/en/publicati...,Jan 2023
3,4,Sally Abbott,Variation and outcomes of liver-reducing dieta...,https://pureportal.coventry.ac.uk/en/publicati...,Feb 2023
4,5,Sally Abbott,Global variations in preoperative practices co...,https://pureportal.coventry.ac.uk/en/publicati...,Jul 2022
5,6,Sally Abbott,Managing idiopathic intracranial hypertension ...,https://pureportal.coventry.ac.uk/en/publicati...,21 Apr 2022
6,7,Sally Abbott,Risk of Harm from Use of Sodium-Glucose Co-Tra...,https://pureportal.coventry.ac.uk/en/publicati...,Jul 2022


In [5]:
single_row = scraped_db.loc[1,:].copy()
single_row

Document_Id                                                       2
Profile_Name                                           Sally Abbott
Title             Emotional eating among adults with healthy wei...
Title_Link        https://pureportal.coventry.ac.uk/en/publicati...
Year_Published                                           3 Apr 2023
Name: 1, dtype: object

# 2.1 Preprocess Text

In [6]:
# Remove stop words
sw = stopwords.words("english")
lemmatizer = WordNetLemmatizer()




def tp1(txt):
    txt = txt.lower()   # Make lowercase
    txt = txt.translate(str.maketrans('',
                                      '',
                                      string.punctuation))   # Remove punctuation marks
    txt = lematize(txt)
    return txt


def fwpt(word):
    tag = pos_tag([word])[0][1][0].upper()
    hash_tag = {"V": wordnet.VERB, "R": wordnet.ADV,"N": wordnet.NOUN,"J": wordnet.ADJ}         
    return hash_tag.get(tag, wordnet.NOUN)

def lematize(text):
        tkns = nltk.word_tokenize(text)
        ax = ""
        for each in tkns:
            if each not in sw:
                ax += lemmatizer.lemmatize(each, fwpt(each)) + " "
        return ax

In [7]:
# Sample title
single_row['Title']

'Emotional eating among adults with healthy weight, overweight and obesity: a systematic review and meta‐analysis'

In [8]:
# Demonstration of lowercase and punctuation removal
tp1(single_row['Title'])

'emotional eat among adult healthy weight overweight obesity systematic review meta‐analysis '

In [9]:
# Demonstration of lematization

lematize(tp1(single_row['Title']))
#lematize(single_row['Title'])

'emotional eat among adult healthy weight overweight obesity systematic review meta‐analysis '

In [10]:
sample_db['Title'].iloc[5]

'Managing idiopathic intracranial hypertension in pregnancy: practical advice'

In [11]:
scraped_db['Title'].iloc[5]

'Managing idiopathic intracranial hypertension in pregnancy: practical advice'

## 2.1.1 Preprocess entire dataframe

In [12]:
processed_db = scraped_db.copy()

def preprocess_df(df):
    df.Title = df.Title.apply(tp1)
    df.Profile_Name = df.Profile_Name.str.lower()
    df = df.drop(columns=['Profile_Name','Year_Published'], axis=1)
    return df
    
preprocess_df(processed_db)
processed_db.head()

,Document_Id,Profile_Name,Title,Title_Link,Year_Published
0,1,aliano abbasi,adopt industry 40 technology citizen ’ electro...,https://pureportal.coventry.ac.uk/en/publicati...,18 Apr 2020
1,2,sally abbott,emotional eat among adult healthy weight overw...,https://pureportal.coventry.ac.uk/en/publicati...,3 Apr 2023
2,3,sally abbott,time throw weigh scale implicit weight bias am...,https://pureportal.coventry.ac.uk/en/publicati...,Jan 2023
3,4,sally abbott,variation outcome liverreducing dietary regime...,https://pureportal.coventry.ac.uk/en/publicati...,Feb 2023
4,5,sally abbott,global variation preoperative practice concern...,https://pureportal.coventry.ac.uk/en/publicati...,Jul 2022


# 2.2 Index Construction

In [13]:
single = processed_db.loc[0,:].copy()
print(single)
indexing_trial = {}

words = single.Title.split()
Document_Id = single.Document_Id
word = words[0]
example = {word: [Document_Id]}

print('=====================================================================')
print('Sample index')
print(example)

Document_Id                                                       1
Profile_Name                                          aliano abbasi
Title             adopt industry 40 technology citizen ’ electro...
Title_Link        https://pureportal.coventry.ac.uk/en/publicati...
Year_Published                                          18 Apr 2020
Name: 0, dtype: object
Sample index
{'adopt': [1]}


In [14]:
## Indexer Function
def apply_index(inputs, index):
    words = inputs.Title.split()
    Document_Id = int(inputs.Document_Id)
    for word in words:
        if word in index.keys():
            if Document_Id not in index[word]:
                index[word].append(Document_Id)
        else:
            index[word] = [Document_Id]
    return index

indx = apply_index(inputs=single, index= {})
print(indx)

{'adopt': [1], 'industry': [1], '40': [1], 'technology': [1], 'citizen': [1], '’': [1], 'electronicengagement': [1], 'consider': [1], 'sustainability': [1], 'development': [1]}


In [15]:
def full_index(df, index):
    for x in range(len(df)):
        inpt = df.loc[x,:]
        ind = apply_index(inputs=inpt, index=index)
    return ind

def construct_index(df, index):
    queue = preprocess_df(df)
    ind = full_index(df=queue, index=index)
    return ind

indexed = full_index(processed_db, 
                     index = {})


indexes = construct_index(df=scraped_db, 
                          index = {})

In [16]:
with open('indexes.json', 'w') as new_f:
    json.dump(indexes, new_f, sort_keys=True, indent=4)
    
with open('indexes.json', 'r') as file:
    data = json.load(file)

def index_2(df, x_path):
    if len(df) > 0:
        with open(x_path, 'r') as file:
            prior_index = json.load(file)
        new_index = construct_index(df = df, index = prior_index)
        with open(x_path, 'w') as new_f:
            json.dump(new_index, new_f, sort_keys=True, indent=4)

In [17]:
len(data)

20743

In [18]:
data

{'0': [8900, 13361],
 '00': [13257],
 '002': [10195, 10200],
 '018': [10195, 10200],
 '01nb': [10168],
 '01o': [10168],
 '02': [10185, 10187],
 '021008': [18475],
 '025': [5745, 8521, 10195],
 '02o': [10165],
 '02o2δ': [10198],
 '02o2−': [10199],
 '03': [10188],
 '04': [10185, 10187, 10200],
 '05': [10054, 10195],
 '054116': [14266],
 '06': [8763, 10185, 10187, 10200],
 '075': [10195],
 '08': [10195, 10200],
 '08cu': [10168],
 '08sm': [10165],
 '1': [26,
  162,
  170,
  176,
  230,
  745,
  804,
  891,
  1251,
  1725,
  1963,
  2457,
  3466,
  3468,
  3714,
  3822,
  3824,
  3840,
  4134,
  4137,
  4279,
  4280,
  4557,
  4561,
  4569,
  4750,
  4753,
  4854,
  5093,
  6185,
  6504,
  6508,
  6988,
  7017,
  7229,
  7474,
  7621,
  7625,
  8082,
  8085,
  8133,
  8174,
  8845,
  9077,
  9584,
  10066,
  10092,
  10185,
  10277,
  11011,
  11144,
  11486,
  11500,
  11577,
  11584,
  11862,
  12025,
  14448,
  14827,
  14843,
  14878,
  15068,
  15177,
  15216,
  15419,
  15542,
  15920

# 3. Query Processor

In [19]:
def demonstrate_query_processing():
    sample = input('Enter Search Terms: ')
    processed_query = tp1(sample)
    #print(f'User Search Query: {sample}')
    print(f'Processed Search Query: {processed_query}')
    return processed_query
    
#demonstrate_query_processing()

# 3.1. Split Query into individual terms

In [20]:
def split_query(terms):
    each = tp1(terms)
    return each.split()

dqp = demonstrate_query_processing()
dqp
print(f'Split Query: {split_query(dqp)}')

Enter Search Terms: python
Processed Search Query: python 
Split Query: ['python']


# 3.2. Boolean Functionalities

In [21]:
def union(lists):
    union = list(set.union(*map(set, lists)))
    union.sort()
    return union

def intersection(lists):
    intersect = list(set.intersection(*map(set, lists)))
    intersect.sort()
    return intersect

# 3.3. Search Engine Function

In [22]:
def vertical_search_engine(df, query, index=indexes):
    query_split = split_query(query)
    retrieved = []
    for word in query_split:
        if word in index.keys():
            retrieved.append(index[word])
            
            
    # Ranked Retrieval
    if len(retrieved)>0:
        high_rank_result = intersection(retrieved)
        low_rank_result = union(retrieved) 
        c = [x for x in low_rank_result if x not in high_rank_result]      
        high_rank_result.extend(c)
        result = high_rank_result
        
        final_output = df[df.Document_Id.isin(result)].reset_index(drop=True)
    
        # Return result in order of Intersection ----> Union
        dummy = pd.Series(result, name = 'Document_Id').to_frame()
        result = pd.merge(dummy, final_output, on='Document_Id', how = 'left')
        
    else:
        result = 'No result found'
    
    return result

In [23]:
def test_search_engine():
    xtest = scraped_db.copy()
    query = input("Enter your search query: ")
    return vertical_search_engine(xtest, query, indexed)
    
test_search_engine()

Enter your search query: python


,Document_Id,Profile_Name,Title,Title_Link,Year_Published
0,1610,christophe bastien,minimum area discrepancy method madm python ex...,https://pureportal.coventry.ac.uk/en/publicati...,28 Jan 2022
1,3361,jesper christensen,minimum area discrepancy method madm python ex...,https://pureportal.coventry.ac.uk/en/publicati...,28 Jan 2022
2,5468,matthew england,compute coderunner coventry university automat...,https://pureportal.coventry.ac.uk/en/publicati...,13 Jan 2020
3,11796,faye mitchell,pilet interactive learn tool teach python,https://pureportal.coventry.ac.uk/en/publicati...,2015
4,15898,ali shafiq,chinese consumer elearning satisfaction contin...,https://pureportal.coventry.ac.uk/en/publicati...,13 Jun 2022


In [24]:
def final_engine(results):
    if type(results) != 'list':
        return results
        #print(results)
    else:
        for i in range(len(results)):
            printout = results.loc[i, :]
            #print(printout['Title'])
            #print(printout['Author'])
            #print(printout['Published'])
            #print(printout['Link'])
            #print('')

In [25]:
scraped_db['Title'].iloc[24]

'response glp1 receptor agonist treatment prebariatric surgery predict postsurgery weight loss patient type 2 diabetes retrospective study '

In [26]:
final_engine(test_search_engine())

Enter your search query: final python


,Document_Id,Profile_Name,Title,Title_Link,Year_Published
0,1610,christophe bastien,minimum area discrepancy method madm python ex...,https://pureportal.coventry.ac.uk/en/publicati...,28 Jan 2022
1,1626,christophe bastien,road safety trust rst 6532017 “ reduce road tr...,https://pureportal.coventry.ac.uk/en/publicati...,22 May 2020
2,1711,jason begley,urbanrural connectivity nonmetropolitan region...,https://pureportal.coventry.ac.uk/en/publicati...,5 Jun 2019
3,1945,nigel berkeley,ressi regional strategy sustainable inclusive ...,https://pureportal.coventry.ac.uk/en/publicati...,1 Feb 2018
4,1981,nigel berkeley,location independent work academic pilot proje...,https://pureportal.coventry.ac.uk/en/publicati...,2009
5,1983,nigel berkeley,canley regeneration project household survey 2...,https://pureportal.coventry.ac.uk/en/publicati...,2008
6,2434,faouzi bouali,keith briggs editor “ d44 final definition rm ...,https://pureportal.coventry.ac.uk/en/publicati...,Nov 2017
7,2653,kevin broughton,evaluation learn prosper business support proj...,https://pureportal.coventry.ac.uk/en/publicati...,12 Sept 2018
8,2655,kevin broughton,ressi regional strategy sustainable inclusive ...,https://pureportal.coventry.ac.uk/en/publicati...,1 Feb 2018
9,2668,kevin broughton,west midland erdf programme 20072013 final eva...,https://pureportal.coventry.ac.uk/en/publicati...,2012


# 4. Schedule Crawler for every week

In [27]:
# days = 0
# interval = 7
# while days <= 1:
#     scrape(mx)
#     print(f"Crawled at {datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
#     print(f'Next crawl scheduled after {interval} days')
#     time.sleep(interval)
#     days = days + 1

# 5. GUI

In [28]:
from tkinter import *
from PIL import Image, ImageTk 
from tkinter import scrolledtext
from pandastable import Table, TableModel
from contextlib import suppress
import warnings
import tkinter.messagebox
warnings.filterwarnings('ignore')


image1 = Image.open('Softwarica-logo.png')
resized_image1 = image1.resize((500,300), Image.ANTIALIAS)

def new_gui(pic):
    window = Tk()
    window.title("Coventry University Oracle")
    window.geometry('1100x600')
    
    img = ImageTk.PhotoImage(pic)
    
    lbl = Label(image=img)
    lbl.image = img
    lbl.grid(column=1, row=3, padx=5, pady=5)
    
    lbl2 = Label(window, text="CEM Search Engine", font=("Arial Bold", 30), padx=5, pady=5)
    lbl.grid(column=1, row=0)
    
    lbl3 = Label(window, text="Enter your search query here ===>", font=("Arial", 15), padx=5, pady=5)
    lbl3.grid(column=0, row=1)
    
    
    query = Entry(window,width=60)
    query.grid(column=1, row=1,  padx=5, pady=5)
    
    results = Canvas(window, height=30, width=250)
    results.grid(column=1, row=2, padx=5, pady=5)
    
    # Entry
    def getInputBoxValue():
        userInput = query.get()
        return userInput

    
    # Button
    def clicked():
        search()
        pass
        
    def no_result():
        tkinter.messagebox.showwarning("Warning", "No results found. Please try different search terms")
        
    
    def search():
        xtest = scraped_db.copy()
        q = query.get()
        f = Frame(window)
        df = vertical_search_engine(xtest, q, indexed)
        if type(df) == str:
            no_result()
        else:
            pt = Table(results)
            try:
                table = pt = Table(results, dataframe=df)
                pt.show()
            except AttributeError:
                pass
            
    def close_window():
        if tkinter.messagebox.askokcancel("Quit", "Quit Programme?"):
            window.destroy()
        
    
    btn = Button(window, text="Search", command=clicked)
    btn.grid(column=2, row=1)
    
    

    window.protocol("WM_DELETE_WINDOW", close_window)       
    window.mainloop()

In [29]:
new_gui(image1)